# Aumento de datos (Text Augmentation)

### Objetivo
En este notebook vamos a aplicar técnicas de data augmentation sobre los comentarios tóxicos
del dataset original, con el objetivo de aumentar el número de ejemplos de la clase 1 (IsToxic)
y mejorar la capacidad de generalización del modelo.

En esta primera fase, usamos la técnica de reemplazo de sinónimos con WordNet a través
de la librería NLPaug. Esto genera versiones modificadas de los textos tóxicos manteniendo
su significado general, permitiendo al modelo aprender con más ejemplos variados.


In [ ]:

import pandas as pd
import nltk
import nlpaug.augmenter.word as naw

# Descargar recursos de WordNet
nltk.download('wordnet')
nltk.download('omw-1.4')

# Cargar dataset original
df = pd.read_csv("../data/youtoxic_english_1000.csv")
df_toxic = df[df["IsToxic"] == 1].copy()

# 🔁 Augmentation con sinónimos (WordNet)
syn_aug = naw.SynonymAug(aug_src='wordnet')
df_toxic["augmented"] = df_toxic["Text"].apply(lambda x: syn_aug.augment(x))

# 👀 Visualizar ejemplos
def mostrar_ejemplos(df_tox, n=5):
    for i in range(n):
        print(f"\n🔹 ORIGINAL: {df_tox.iloc[i]['Text']}")
        print(f"🔸 AUGMENTED: {df_tox.iloc[i]['augmented']}")

mostrar_ejemplos(df_toxic)

# Unir datos originales + aumentados
df_aug = pd.DataFrame({
    "Text": df_toxic["augmented"],
    "IsToxic": 1
})

df_full = pd.concat([df, df_aug], ignore_index=True)

# Verificar balance de clases
print("\n✅ Recuento por clase tras augmentación:")
print(df_full["IsToxic"].value_counts())

# Guardar nuevo dataset para reentrenamiento posterior
df_full.to_csv("../data/youtoxic_augmented.csv", index=False)
print("\n💾 Dataset aumentado guardado como 'youtoxic_augmented.csv'")


In [ ]:
from deep_translator import GoogleTranslator
import time

# Cargamos el dataset aumentado
df_augmented = pd.read_csv("../data/youtoxic_augmented.csv")

# Filtramos solo los tóxicos
df_toxic = df_augmented[df_augmented["IsToxic"] == 1].copy()

# Función de back translation: Inglés ➝ Español ➝ Inglés
def back_translate(text):
    try:
        spanish = GoogleTranslator(source='en', target='es').translate(text)
        english = GoogleTranslator(source='es', target='en').translate(spanish)
        time.sleep(0.5)  # para evitar límite de Google
        return english
    except Exception as e:
        print(f"Error con el texto: {text[:50]}... ➝ {e}") 
        return text

# Aplicamos la función
df_toxic["back_translated"] = df_toxic["Text"].apply(back_translate)


In [ ]:
# Creamos DataFrame con los nuevos ejemplos
df_back = pd.DataFrame({
    "Text": df_toxic["back_translated"],
    "IsToxic": 1
})

# Unimos al dataset ya aumentado
df_final = pd.concat([df_augmented, df_back], ignore_index=True)

# Guardamos la nueva versión ampliada
df_final.to_csv("../data/youtoxic_augmented_backtranslated.csv", index=False)
print("💾 Dataset final guardado como 'youtoxic_augmented_backtranslated.csv'")
